In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA

# Load data
df = pd.read_csv('/content/heart.csv')

# Identify categorical columns
binary_cols = ['Sex', 'ExerciseAngina']  # M/F, Y/N
multi_cols = ['ChestPainType', 'RestingECG', 'ST_Slope']  # More than 2 categories

# Label encode binary columns
le = LabelEncoder()
for col in binary_cols:
    df[col] = le.fit_transform(df[col])

# One-hot encode multi-class columns
df = pd.get_dummies(df, columns=multi_cols)

# Split features and target
X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define models
models = {
    'SVM': SVC(random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42)
}

print("Model Performance WITHOUT PCA:")
best_acc = 0
best_model = None

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name}: Accuracy = {acc:.4f}")
    if acc > best_acc:
        best_acc = acc
        best_model = name
    if name == 'Random Forest':  # Example: Show confusion matrix for RF
        print("Confusion Matrix (Random Forest):")
        print(confusion_matrix(y_test, y_pred))

# Apply PCA
pca = PCA(n_components=0.95, random_state=42)  # Keep 95% variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print("\nModel Performance WITH PCA:")
for name, model in models.items():
    model.fit(X_train_pca, y_train)
    y_pred = model.predict(X_test_pca)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} (PCA): Accuracy = {acc:.4f}")

print(f"\nBest model without PCA: {best_model} (Accuracy: {best_acc:.4f})")
print(f"Number of PCA components: {pca.n_components_}")


Model Performance WITHOUT PCA:
SVM: Accuracy = 0.8967
Logistic Regression: Accuracy = 0.8859
Random Forest: Accuracy = 0.8750
Confusion Matrix (Random Forest):
[[69 13]
 [10 92]]

Model Performance WITH PCA:
SVM (PCA): Accuracy = 0.8913
Logistic Regression (PCA): Accuracy = 0.8859
Random Forest (PCA): Accuracy = 0.8967

Best model without PCA: SVM (Accuracy: 0.8967)
Number of PCA components: 14
